___
# Import Library
___

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import category_encoders as ce

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_squared_error

from dotenv import load_dotenv, find_dotenv, set_key
import sqlalchemy as db
import os
import pickle

In [2]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [3]:
load_dotenv()

True

In [4]:
RANDOM_STATE = 202102
SCORING = 'neg_root_mean_squared_error'

___
# Business Problems
___

Who doesn't like to travel? Traveling is a such fun activity, coming into new place and experience many new things. It is a part of crucial aspect in life as it's the best way to escape from hectic and busy schedule. It also improves the mental and physical health while it's a good remedy for stress, anxiety and depression. As part of travelling, deciding accomodation or where you should sleep and charge your body is definitely important thing to decide. One of the choice to spend your night, apart from conventional hotel is Airbnb.

Airbnb is an American vacation rental online marketplace company based in San Francisco, California. Airbnb maintains and hosts a marketplace, accessible to consumers on its website or via an app. Through the service, users can arrange lodging, primarily homestays, and tourism experiences or list their properties for rental. Airbnb does not own any of the listed properties; instead, it profits by receiving commission from each booking. 

Singapore, one of developed country in Asia, make 4.1% of its national GDP from tourism industry alone and has been one of the most visited country in Asia resulting in thousand of Airbnb listings in Singapore (around 4000+ listings). It can be troublesome to choose one befitting your needs and budget. This also becomes problem for owner property who wants to register their new property in Airbnb since they may find it hard to price their property.

Here's where machine learning help to solve those problems. Predicting price of property, or in this case Airbnb listing, could be quite challenging since there are various factors need to be measured and calculated in order to get price accurately. In this project, we will do end-to-end machine learning project, starting from cleaning the dataset, do exploratory data analysis to get some insights, use machine learning model to predict Airbnb listing price and deploy our best model by creating dashboard.   

___
# Load Dataset
___

In [5]:
engine = db.create_engine(os.getenv('db-uri'))

In [6]:
meta = db.MetaData()
meta.reflect(engine)

In [7]:
with engine.connect() as con:
    query = db.select([meta.tables['listings']])
    result = con.execute(query).fetchall()
df_raw = pd.DataFrame(result, columns=meta.tables['listings'].c.keys())

In [8]:
listings_dropped_columns = os.getenv('listings_dropped_columns')
listings_dropped_columns

"['id', 'listing_url', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month', 'host_response_rate', 'host_acceptance_rate', 'host_listings_count', 'host_total_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'availability_60', 'availability_90', 'availability_365', 'host_i

In [9]:
df = df_raw.drop(eval(listings_dropped_columns), axis=1)
display(df)

neighbourhood_cleansed neighbourhood_group_cleansed  \
0                 Woodlands                 North Region   
1               Bukit Timah               Central Region   
2                 Woodlands                 North Region   
3                  Tampines                  East Region   
4                  Tampines                  East Region   
...                     ...                          ...   
4382            Bukit Timah               Central Region   
4383          Marine Parade               Central Region   
4384                Geylang               Central Region   
4385          Marine Parade               Central Region   
4386                Kallang               Central Region   

                    property_type        room_type  accommodates  bedrooms  \
0       Private room in apartment     Private room             1       1.0   
1       Private room in apartment     Private room             2       1.0   
2       Private room in apartment     Private room             1       1.0   
3           Private room in villa     Private room             6       2.0   
4           Private room in house     Private room             3       1.0   
...                           ...              ...           ...       ...   
4382  Private room in condominium     Private room             2       NaN   
4383        Private room in house     Private room             1       1.0   
4384        Private room in house     Private room             4       1.0   
4385        Private room in house     Private room             1       1.0   
4386             Entire apartment  Entire home/apt             2       NaN   

      beds  price  minimum_nights  maximum_nights  availability_30  \
0      1.0     79             180             360               30   
1      1.0     80              90             730               30   
2      1.0     66               6              14               30   
3      3.0    174              90            1125               30   
4      1.0     93              90            1125               30   
...    ...    ...             ...             ...              ...   
4382   1.0     25              10              30                0   
4383   1.0     57              90             182               29   
4384   1.0     62              90             182               29   
4385   1.0     47              90             182               29   
4386   1.0    160               6            1125               29   

      instant_bookable  calculated_host_listings_count  total_bathrooms  \
0                False                               2              1.0   
1                False                               1              1.0   
2                False                               2              1.0   
3                False                               8              1.0   
4                False                               8              1.0   
...                ...                             ...              ...   
4382              True                               2              1.0   
4383              True                               6              1.0   
4384              True                               6              1.0   
4385              True                               6              1.0   
4386             False                             126              1.0   

        bathrooms_type  Air Conditioning  BBQ Utensils  \
0                 bath                 1             0   
1                 bath                 1             0   
2                 bath                 1             0   
3         private bath                 1             0   
4     shared half-bath                 1             0   
...                ...               ...           ...   
4382       shared bath                 1             0   
4383         half-bath                 1             0   
4384              bath                 1             0   
4385         half-bath   

___
# Data Splitting
___

In [10]:
X = df.drop('price', axis=1).fillna(np.nan)
y = df['price']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=RANDOM_STATE)

In [12]:
for x in [X_train, X_test, y_train, y_test]:
    print(x.shape)

(3509, 64)
(878, 64)
(3509,)
(878,)


In [13]:
list_column_input = X_train.columns.tolist()
if os.getenv('list_column_input') != list_column_input:
    set_key(find_dotenv(), 'list_column_input', str(list_column_input))

___
# Choose Evaluation Metric
___

In this project, one of our goal is to predict Airbnb listing as accurately as possible (regression problem). Hence, I choose MSE (Mean Squared Error) as evaluation metric since it shows how far our set of predictions to the actual price and its ability to punish high difference of prediction and actual price.

___
# Data Transformer
___

In [14]:
encode_df = pd.DataFrame()
encode_df['Unique Values'] = df.apply(pd.unique)
encode_df['Count'] = encode_df['Unique Values'].apply(len)
encode_df['Data Type'] = [df[t].dtype for t in encode_df.index]
encode_df['%Missing'] = df.isnull().sum()/len(df)*100
encode_df.sort_values('Count')

Unique Values  \
Entertainment                                                                    [0, 1]   
Ethernet Connection                                                              [1, 0]   
Extra pillows, blankets or bed-linen                                             [0, 1]   
Fan                                                                              [0, 1]   
Fire Safety Equipments                                                           [0, 1]   
Fireplace                                                                        [0, 1]   
First Aid Kit                                                                    [0, 1]   
Free Parking                                                                     [0, 1]   
Gym                                                                              [0, 1]   
Heating                                                                          [0, 1]   
Host Greets You                                                                  [0, 1]   
Hot Tub                                                                          [0, 1]   
Hot Water                                                                        [0, 1]   
Kitchen Utensils                                                                 [0, 1]   
Kitchen/Dining Area                                                              [0, 1]   
Laundromat Nearby                                                                [0, 1]   
Living Room                                                                      [0, 1]   
Lockbox                                                                          [0, 1]   
Long Term Stays Allowed                                                          [0, 1]   
Luggage Dropoff Allowed                                                          [0, 1]   
Outdoor Space                                                                    [0, 1]   
Paid Parking                                                                     [0, 1]   
Pool                                                                             [0, 1]   
Private Entrance                                                                 [0, 1]   
Refrigerator/Freezer                                                             [0, 1]   
Sauna                                                                            [0, 1]   
Single level Home                                                                [0, 1]   
Ski-in/Ski-out                                                                   [0, 1]   
TV                                                                               [1, 0]   
Toilet Equipments                                                                [0, 1]   
Essentials                                                                       [0, 1]   
Washer                                                                           [1, 0]   
Water Body Access                                                                [0, 1]   
EV Charger                                                                       [0, 1]   
instant_bookable                                                          [False, True]   
Elevator                                                                         [1, 0]   
BBQ Utensils                                                                     [0, 1]   
Baby and Children Equipments                                                     [0, 1]   
Bathtub                                                                          [0, 1]   
Beach Essentials                                                                 [0, 1]   
Breakfast                                                                        [0, 1]   
Building Staff                                                                   [0, 1]   
Air Conditioning                                                                 [1, 0]   
Cleaning Equipments                                                   

In [15]:
missing_numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=np.nan))
])

In [16]:
missing_onehot_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=np.nan)),
    ('onehot', OneHotEncoder(drop='first'))
])

In [17]:
transformer = ColumnTransformer([
    ('missing_numerical', missing_numerical_pipeline, ['bedrooms', 'beds', 'total_bathrooms']),
    ('missing_onehot', missing_onehot_pipeline, ['bathrooms_type']), 
    ('nonmissing_onehot', OneHotEncoder(drop='first'), ['instant_bookable', 
                                                        'room_type', 'neighbourhood_group_cleansed']),
    ('nonmissing_binary', ce.BinaryEncoder(drop_invariant=True), ['neighbourhood_cleansed', 
                                                                  'property_type'])
], remainder='passthrough')

___
# Model Building
___

In [41]:
model_result = pd.DataFrame()

In [19]:
kf = KFold(shuffle=True, random_state=RANDOM_STATE)

## Random Forest Regression

### Benchmark Random Forest Regression

In [20]:
rfr = RandomForestRegressor(random_state=RANDOM_STATE)

In [21]:
benchmark_rfr = Pipeline([
    ('transformer', transformer), 
    ('regressor', rfr)
])

In [22]:
benchmark_rfr_scores = cross_val_score(benchmark_rfr, X_train, y_train, scoring=SCORING, 
                                       cv=kf, n_jobs=-1, verbose=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.8min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:  1.8min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.9min finished


In [23]:
benchmark_rfr_scores

array([-425.82050306, -128.6600083 , -350.57105365, -337.1799072 ,
       -286.4905963 ])

In [24]:
benchmark_rfr_scores.mean()

-305.74441370388865

In [25]:
benchmark_rfr.fit(X_train, y_train)

c:\users\zylveyn\appdata\local\programs\python\python39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('missing_numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['bedrooms', 'beds',
                                                   'total_bathrooms']),
                                                 ('missing_onehot',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['bathrooms_type']),
   

In [26]:
filename = 'benchmark_rfr.pkl'
pickle.dump(benchmark_rfr, open('Models/Trained Models/'+filename, 'wb'))

In [42]:
model_result = model_result.append(pd.DataFrame.from_dict({'Benchmark RFR': -benchmark_rfr_scores.mean()}, 
                                                          orient='index'))
model_result

0
Benchmark RFR  305.744414

### Tuned Random Forest Regression

In [28]:
rfr = RandomForestRegressor(random_state=RANDOM_STATE)

In [29]:
benchmark_rfr = Pipeline([
    ('transformer', transformer), 
    ('regressor', rfr)
])

In [30]:
tuned_rfr_params = {
    'regressor__n_estimators': [100, 200],
    'regressor__criterion': ['mse','mae'],
    'regressor__max_depth': [10, 20, 50],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [2, 5, 10]
}

In [31]:
grid_rfr = GridSearchCV(benchmark_rfr, tuned_rfr_params, scoring=SCORING, 
                        cv=kf, n_jobs=-1, verbose=10)

In [32]:
grid_rfr.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


c:\users\zylveyn\appdata\local\programs\python\python39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


GridSearchCV(cv=KFold(n_splits=5, random_state=202102, shuffle=True),
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('missing_numerical',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent'))]),
                                                                         ['bedrooms',
                                                                          'beds',
                                                                          'total_bathrooms']),
                                                                        ('missing_onehot',
                                                                         Pipeline(steps=[('imputer',
        

In [33]:
grid_rfr.best_score_

-294.1641540293723

In [34]:
tuned_rfr_scores = pd.DataFrame(grid_rfr.cv_results_).sort_values('rank_test_score')
tuned_rfr_scores

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
29        8.155330      0.307110         0.107740        0.012456   
25        8.099962      0.147913         0.121131        0.013159   
27        7.805931      0.215813         0.104941        0.005326   
43        7.598849      0.051041         0.104141        0.003309   
45        7.670609      0.029829         0.103741        0.003308   
47        7.650420      0.142614         0.105940        0.005582   
28        4.336118      0.214356         0.071359        0.005532   
24        4.089859      0.248638         0.072159        0.006109   
26        4.140030      0.093551         0.079955        0.008457   
42        3.898967      0.027980         0.070361        0.002869   
44        3.899367      0.028173         0.072762        0.004532   
46        3.859191      0.051202         0.069162        0.001166   
21        9.163553      0.049640         0.109738        0.010919   
19        9.764010      0.189239         0.115535        0.008633   
9         6.260010      0.057300         0.097346        0.005423   
7         6.409529      0.171387         0.094148        0.002558   
11        6.249222      0.042253         0.095147        0.006043   
39        9.858356      0.206549         0.123530        0.017570   
8         3.178445      0.021623         0.067562        0.002058   
6         3.327292      0.137785         0.074558        0.009347   
10        3.231548      0.026614         0.071160        0.006043   
37        9.729630      0.029985         0.107540        0.001854   
95      206.757426      5.858552         0.121731        0.041175   
36        4.874609      0.043595         0.076956        0.003161   
20        4.700110      0.015410         0.073357        0.003437   
57      191.035828      3.052801         0.084351        0.000800   
93      204.699804      1.427983         0.097145        0.001719   
75      221.627948      7.810407         0.094546        0.003260   
18        4.924581      0.138584         0.083553        0.007860   
55      195.066120      1.549664         0.084152        0.001165   
77      228.639898      3.227251         0.128327        0.019261   
3         7.236456      0.094889         0.116734        0.017188   
59      191.317666      1.605013         0.084752        0.001166   
1         6.697565      0.106440         0.103741        0.011314   
38        4.783461      0.103429         0.073359        0.003927   
91      204.069365      1.535706         0.095945        0.001264   
74      103.276471      3.280935         0.071760        0.018094   
2         3.481007      0.070534         0.072159        0.005561   
54       97.709059      2.213379         0.061565        0.002576   
92      102.533097      1.363760         0.066962        0.005583   
73      205.771191      2.138962         0.098344        0.004406   
56       96.992869      1.033346         0.058167        0.000747   
94      101.047947      0.654171         0.061366        0.001743   
76      106.165018      3.837247         0.066162        0.011438   
58       96.214115      1.275681         0.061565        0.002153   
0         3.383742      0.058949         0.072759        0.005876   
41        8.472349      0.062846         0.103341        0.003381   
83      183.755395      1.893186         0.093347        0.005459   
81      189.780433      1.832592         0.107139        0.036015   
79      188.874864      3.142835         0.092147        0.010413   
101     208.281554      3.817889         0.114335        0.026740   
99      187.634174      9.795542         0.089349        0.002575   
97      186.841828      6.483260         0.096745        0.009698   
23        8.847534      0.096901         0.130926        0.026321   
90      102.748773      0.790947         0.067362        0.005459   
100     105.901170      6.943208         0.068759        0.007246   
98       96.899322      5.286986         0.071160        0.024890   
96 

In [35]:
tuned_rfr = grid_rfr.best_estimator_

In [36]:
tuned_rfr.fit(X_train, y_train)

c:\users\zylveyn\appdata\local\programs\python\python39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('missing_numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['bedrooms', 'beds',
                                                   'total_bathrooms']),
                                                 ('missing_onehot',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['bathrooms_type']),
   

In [37]:
filename = 'tuned_rfr.pkl'
pickle.dump(tuned_rfr, open('Models/Trained Models/'+filename, 'wb'))

In [43]:
model_result = model_result.append(pd.DataFrame.from_dict({'Tuned RFR': -grid_rfr.best_score_}, 
                                                          orient='index'))
model_result

0
Benchmark RFR  305.744414
Tuned RFR      294.164154

### Save Model Result

In [49]:
filename = 'model_result_rfr.pkl'
pickle.dump(model_result, open('Models/Trained Models/'+filename, 'wb'))